<h1>Trabalho 1 de SAD</h1>

<h3>1 - Posse de bola influencia no resultado final da partida?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_team_goal', 'away_team_goal', 'possession'

<h3>2 - Time que menos modifica seus jogadores tem melhor colocacao na tabela?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_player[1..11]', 'match_api_id', 'session', 'home_team_api_id'\
       'away_team_api_id', 'away_player[1..11]', 'home_team_goal', 'away_team_goal'

<h3>3 - Time que mais chuta ao gol influencia no resultado final da partida?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_team_goal', 'away_team_goal', 'shoton', 'shotoff'

In [134]:
import pandas as pd
import numpy as np
import os
import seaborn as sb
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

%matplotlib inline

path_banco = os.getenv("HOME") + '/Bancos_Dados/UFJF/Periodo_2019_1/SAD/'

<h2>Bases de Dados</h2>

In [104]:
#Bases

#base_Country = pd.read_csv(path_banco + 'Country.csv')
#base_League = pd.read_csv(path_banco + 'League.csv')
base_Match_Total = pd.read_csv(path_banco + 'Match.csv',low_memory = False)
#base_Player = pd.read_csv(path_banco + 'Player.csv')
#base_Player_Attributes = pd.read_csv(path_banco + 'Player_Attributes.csv')
#base_Team= pd.read_csv(path_banco + 'Team.csv')
#base_Team_Attributes= pd.read_csv(path_banco + 'Team_Attributes.csv')
#base_sqlite_sequence= pd.read_csv(path_banco + 'sqlite_sequence.csv')


<h2>Pegando as colunas necessárias</h2>

In [105]:
#Pegando somente as colunas necessarias
colunas_em_uso = ['id','country_id','league_id','season','home_team_goal','away_team_goal','possession','home_player_1','home_player_2','home_player_3','home_player_4','home_player_5',
                         'home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11',
                         'away_player_1','away_player_2','away_player_3','away_player_4','away_player_5','away_player_6',
                         'away_player_7','away_player_8','away_player_9','away_player_10','away_player_11', 'match_api_id',
                         'home_team_api_id','away_team_api_id','shoton', 'shotoff']
base_Match = base_Match_Total.loc[:,colunas_em_uso]
print('Quantidade de colunas para tratamento:',len(colunas_em_uso))

Quantidade de colunas para tratamento: 34


<h2>Arrumando a base Match</h2>

In [106]:
#Arrumando a base_Match
coluna = base_Match.loc[:,'possession']
coluna1 = base_Match.loc[pd.notnull(base_Match['possession'])]
coluna1 = coluna1.loc[:,['possession']]
coluna2 = coluna1.values

matriz = [] #Matriz que armazena os vetores resultados de cada parser
for linha in coluna:
    soup = BeautifulSoup(str(linha), 'html.parser')
    awaypos = soup.find_all('awaypos')
    homepos = soup.find_all('homepos')

    awaypos_num = []
    for pos in awaypos:
        awaypos_num.append(pos.get_text())
    homepos_num = []
    for pos in homepos:
        homepos_num.append(pos.get_text())
    matriz.append([homepos_num,awaypos_num])
vetor_home = []
vetor_away = []
for vetor in matriz:
    resultado_home = 0
    resultado_away = 0
    if len(vetor[0]) != 0:
        resultado_home = int(vetor[0][len(vetor[0])-1])
    if len(vetor[1]) != 0:
        resultado_away = int(vetor[1][len(vetor[1])-1])
    vetor_home.append(resultado_home)
    vetor_away.append(resultado_away)
#A ideia aqui eh criar duas colunas para possession, onde teremos possession_home e possession_away
#print(matriz[10000])
#Apagando a coluna possession e acrescentando duas colunas: possession_home e possession_away
base_Match.drop('possession', 1, inplace=True)
base_Match['possession_home'] = vetor_home 
base_Match['possession_away'] = vetor_away

In [107]:
#Arrumando o shuton e shutoff
coluna_shoton = base_Match.loc[:, 'shoton'].values
coluna_shotoff = base_Match.loc[:, 'shotoff'].values

shoton = []
for linha in coluna_shoton:
    soup = BeautifulSoup(str(linha), 'html.parser')
    values = soup.find_all('value')
    shoton.append(len(values))

shotoff = []
for linha in coluna_shotoff:
    soup = BeautifulSoup(str(linha), 'html.parser')
    values = soup.find_all('value')
    shotoff.append(len(values))

#print(shoton[10000], shotoff[10000])
#Remover as colunas e adicionar novamente com os valores corretos
base_Match.drop('shoton', 1, inplace = True)
base_Match['shoton'] = shoton
base_Match.drop('shotoff', 1, inplace = True)
base_Match['shotoff'] = shotoff


<h2>Características da Base</h2>

In [108]:
base_Match.describe() #Estatisticas

,id,country_id,league_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,...,away_player_9,away_player_10,away_player_11,match_api_id,home_team_api_id,away_team_api_id,possession_home,possession_away,shoton,shotoff
count,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,24755.000000,24664.000000,24698.000000,24656.000000,24663.000000,...,24651.000000,24538.000000,24425.000000,2.597900e+04,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000
mean,12990.000000,11738.630317,11738.630317,1.544594,1.160938,76638.362432,106854.091996,91601.289457,94540.204494,109527.842071,...,111087.003448,107149.115943,104932.876970,1.195429e+06,9984.371993,9984.475115,16.688672,15.718272,6.917857,6.917703
std,7499.635658,7553.936759,7553.936759,1.297158,1.142110,87556.226324,112619.266880,102703.003558,102789.967205,112787.542376,...,116492.957656,112397.540326,110237.525957,4.946279e+05,14087.453758,14087.445135,24.677657,23.310770,12.622504,12.301140
min,1.000000,1.000000,1.000000,0.000000,0.000000,2984.000000,2802.000000,2752.000000,2752.000000,2752.000000,...,2625.000000,2770.000000,2802.000000,4.831290e+05,1601.000000,1601.000000,0.000000,0.000000,0.000000,0.000000
25%,6495.500000,4769.000000,4769.000000,1.000000,0.000000,30602.000000,32574.000000,30602.000000,30627.000000,33579.000000,...,33435.000000,32627.000000,32747.000000,7.684365e+05,8475.000000,8475.000000,0.000000,0.000000,0.000000,0.000000
50%,12990.000000,10257.000000,10257.000000,1.000000,1.000000,38230.000000,42388.000000,39731.000000,41060.000000,45996.000000,...,45860.000000,45358.000000,42652.000000,1.147511e+06,8697.000000,8697.000000,0.000000,0.000000,0.000000,0.000000
75%,19484.500000,17642.000000,17642.000000,2.000000,2.000000,96836.000000,159854.000000,128036.750000,145561.000000,160243.000000,...,164209.000000,161291.000000,161660.000000,1.709852e+06,9925.000000,9925.000000,45.000000,42.000000,10.000000,11.000000
max,25979.000000,24558.000000,24558.000000,10.000000,9.000000,698273.000000,748432.000000,705484.000000,723037.000000,733787.000000,...,722766.000000,722766.000000,726956.000000,2.216672e+06,274581.000000,274581.000000,83.000000,96.000000,78.000000,75.000000


In [109]:
base_Match.median() #Mediana

id                    12990.0
country_id            10257.0
league_id             10257.0
home_team_goal            1.0
away_team_goal            1.0
home_player_1         38230.0
home_player_2         42388.0
home_player_3         39731.0
home_player_4         41060.0
home_player_5         45996.0
home_player_6         41467.0
home_player_7         41432.0
home_player_8         43319.0
home_player_9         45605.0
home_player_10        43296.0
home_player_11        42091.0
away_player_1         38289.0
away_player_2         42388.0
away_player_3         39892.0
away_player_4         41083.0
away_player_5         46212.0
away_player_6         41634.5
away_player_7         41433.0
away_player_8         45816.0
away_player_9         45860.0
away_player_10        45358.0
away_player_11        42652.0
match_api_id        1147511.0
home_team_api_id       8697.0
away_team_api_id       8697.0
possession_home           0.0
possession_away           0.0
shoton                    0.0
shotoff   

In [110]:
base_Match.columns #Nomes Colunas

Index(['id', 'country_id', 'league_id', 'season', 'home_team_goal',
       'away_team_goal', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11', 'match_api_id',
       'home_team_api_id', 'away_team_api_id', 'possession_home',
       'possession_away', 'shoton', 'shotoff'],
      dtype='object')

In [111]:
base_Match.shape #Linhas x Colunas

(25979, 35)

In [112]:
base_Match.min() #Valor Minimo

id                          1
country_id                  1
league_id                   1
season              2008/2009
home_team_goal              0
away_team_goal              0
home_player_1            2984
home_player_2            2802
home_player_3            2752
home_player_4            2752
home_player_5            2752
home_player_6            2625
home_player_7            2625
home_player_8            2625
home_player_9            2625
home_player_10           2625
home_player_11           2802
away_player_1            2796
away_player_2            2790
away_player_3            2752
away_player_4            2752
away_player_5            2790
away_player_6            2625
away_player_7            2625
away_player_8            2625
away_player_9            2625
away_player_10           2770
away_player_11           2802
match_api_id           483129
home_team_api_id         1601
away_team_api_id         1601
possession_home             0
possession_away             0
shoton    

In [113]:
base_Match.max() #Valor Maximo

id                      25979
country_id              24558
league_id               24558
season              2015/2016
home_team_goal             10
away_team_goal              9
home_player_1          698273
home_player_2          748432
home_player_3          705484
home_player_4          723037
home_player_5          733787
home_player_6          750584
home_player_7          692984
home_player_8          693171
home_player_9          730065
home_player_10         742405
home_player_11         726956
away_player_1          698273
away_player_2          748432
away_player_3          705484
away_player_4          728414
away_player_5          746419
away_player_6          722766
away_player_7          750435
away_player_8          717248
away_player_9          722766
away_player_10         722766
away_player_11         726956
match_api_id          2216672
home_team_api_id       274581
away_team_api_id       274581
possession_home            83
possession_away            96
shoton    

In [114]:
base_Match.count() #Count

id                  25979
country_id          25979
league_id           25979
season              25979
home_team_goal      25979
away_team_goal      25979
home_player_1       24755
home_player_2       24664
home_player_3       24698
home_player_4       24656
home_player_5       24663
home_player_6       24654
home_player_7       24752
home_player_8       24670
home_player_9       24706
home_player_10      24543
home_player_11      24424
away_player_1       24745
away_player_2       24701
away_player_3       24686
away_player_4       24658
away_player_5       24644
away_player_6       24666
away_player_7       24744
away_player_8       24638
away_player_9       24651
away_player_10      24538
away_player_11      24425
match_api_id        25979
home_team_api_id    25979
away_team_api_id    25979
possession_home     25979
possession_away     25979
shoton              25979
shotoff             25979
dtype: int64

<h2>Arrumando valores faltantes e zerados</h2>

In [142]:
#Convertendo a data em inteiro
from sklearn.preprocessing import LabelEncoder
coluna_season = base_Match.iloc[:,3].values
labelencoder_season = LabelEncoder()
coluna_season = labelencoder_season.fit_transform(coluna_season)

base_Match['season'] = coluna_season

In [143]:
colunas_base = base_Match.iloc[:,0:35].values
#Preenchendo os valores faltantes
#strategy = 'mean', 'median', 'most_frequent'
from sklearn.preprocessing import Imputer
#Estrategia do mais frequente
imputer = Imputer(strategy='most_frequent')
imputer = imputer.fit(colunas_base[:, 0:35])
colunas_base[:,0:35] = imputer.transform(colunas_base[:,0:35])



In [144]:
df = pd.DataFrame(colunas_base)
nomes_colunas = base_Match.columns
nomes_colunas_aux = []
for i in nomes_colunas:
    nomes_colunas_aux.append(str(i))
df.set_axis(nomes_colunas_aux, axis='columns', inplace=True)
base_Match = df

In [145]:
base_Match.count() #Conta as colunas que possuem valor faltante

id                  25979
country_id          25979
league_id           25979
season              25979
home_team_goal      25979
away_team_goal      25979
home_player_1       25979
home_player_2       25979
home_player_3       25979
home_player_4       25979
home_player_5       25979
home_player_6       25979
home_player_7       25979
home_player_8       25979
home_player_9       25979
home_player_10      25979
home_player_11      25979
away_player_1       25979
away_player_2       25979
away_player_3       25979
away_player_4       25979
away_player_5       25979
away_player_6       25979
away_player_7       25979
away_player_8       25979
away_player_9       25979
away_player_10      25979
away_player_11      25979
match_api_id        25979
home_team_api_id    25979
away_team_api_id    25979
possession_home     25979
possession_away     25979
shoton              25979
shotoff             25979
dtype: int64

In [154]:
#sb.pairplot(base_Match)

<h2>Separando a base por perguntas</h2>

In [146]:
base1 = base_Match.loc[:,['home_team_goal','away_team_goal','possession_home', 'possession_away']]

base2 = base_Match.loc[:,['country_id','league_id','season','home_player_1','home_player_2','home_player_3','home_player_4','home_player_5',
                         'home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11',
                         'away_player_1','away_player_2','away_player_3','away_player_4','away_player_5','away_player_6',
                         'away_player_7','away_player_8','away_player_9','away_player_10','away_player_11', 'match_api_id',
                         'home_team_api_id','away_team_api_id','home_team_goal','away_team_goal']]

base3 = base_Match.loc[:,['home_team_goal', 'away_team_goal', 'shoton', 'shotoff']]

In [147]:
#base1.mean() #Media
base1.describe()

,home_team_goal,away_team_goal,possession_home,possession_away
count,25979.000000,25979.000000,25979.000000,25979.000000
mean,1.544594,1.160938,16.688672,15.718272
std,1.297158,1.142110,24.677657,23.310770
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000
75%,2.000000,2.000000,45.000000,42.000000
max,10.000000,9.000000,83.000000,96.000000


In [157]:
#sb.pairplot(base1)

In [148]:
#base2.mean() #Media
base2.describe()

,country_id,league_id,season,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,...,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
count,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,...,25979.000000,25979.000000,25979.000000,25979.000000,25979.000000,2.597900e+04,25979.000000,25979.000000,25979.000000,25979.000000
mean,11738.630317,11738.630317,3.496747,74501.916702,103117.025290,88840.214519,91285.376766,105238.173332,99123.404635,99978.325879,...,100577.269833,107506.307248,107268.012857,103187.316332,100631.703761,1.195429e+06,9984.371993,9984.475115,1.544594,1.160938
std,7553.936759,7553.936759,2.306759,86007.023756,110919.036011,100870.059405,101119.368569,111451.665256,109499.157377,105188.428601,...,105460.133329,113565.578183,114663.053909,110452.384939,108241.161776,4.946279e+05,14087.453758,14087.445135,1.297158,1.142110
min,1.000000,1.000000,0.000000,2984.000000,2802.000000,2752.000000,2752.000000,2752.000000,2625.000000,2625.000000,...,2625.000000,2625.000000,2625.000000,2770.000000,2802.000000,4.831290e+05,1601.000000,1601.000000,0.000000,0.000000
25%,4769.000000,4769.000000,1.000000,30660.000000,32833.000000,30740.000000,30627.000000,31291.000000,32421.000000,31013.000000,...,31045.000000,33685.000000,33852.000000,33098.000000,33028.000000,7.684365e+05,8475.000000,8475.000000,1.000000,0.000000
50%,10257.000000,10257.000000,3.000000,37579.000000,41315.000000,39032.000000,39859.000000,43061.000000,40521.000000,42479.000000,...,42554.000000,50189.000000,42237.000000,41424.000000,41171.000000,1.147511e+06,8697.000000,8697.000000,1.000000,1.000000
75%,17642.000000,17642.000000,6.000000,95336.000000,154950.000000,114034.500000,128037.000000,155534.000000,149211.000000,154257.000000,...,154257.000000,155355.000000,158861.000000,154259.000000,154280.000000,1.709852e+06,9925.000000,9925.000000,2.000000,2.000000
max,24558.000000,24558.000000,7.000000,698273.000000,748432.000000,705484.000000,723037.000000,733787.000000,750584.000000,692984.000000,...,750435.000000,717248.000000,722766.000000,722766.000000,726956.000000,2.216672e+06,274581.000000,274581.000000,10.000000,9.000000


In [155]:
#sb.pairplot(base2)

In [149]:
#base3.mean() # Media
base3.describe()

,home_team_goal,away_team_goal,shoton,shotoff
count,25979.000000,25979.000000,25979.000000,25979.000000
mean,1.544594,1.160938,6.917857,6.917703
std,1.297158,1.142110,12.622504,12.301140
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.000000
75%,2.000000,2.000000,10.000000,11.000000
max,10.000000,9.000000,78.000000,75.000000


In [156]:
#sb.pairplot(base3)

In [153]:
#base_Match.to_csv('base_Match_mod.csv')